In [ ]:
!pip install awswrangler nltk gensim

In [16]:
import datetime as dt
from datetime import date, timedelta,datetime
import torch
from torch import nn
import sagemaker
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorchModel
from sagemaker.predictor import RealTimePredictor, json_serializer, json_deserializer
from sagemaker.pytorch import PyTorch as PyTorchEstimator
import transformers
from nltk.tokenize import RegexpTokenizer
from gensim.parsing.preprocessing import strip_punctuation
import os
import awswrangler as wr
import json
import argparse
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
today = date.today()
timedelta(days=0, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
yesterday = today - timedelta(days = 1)
yest = yesterday.strftime("%Y-%m-%d")
data_key = f"daily_comments/df{yest}.csv"

In [7]:
bucket = 's3://bert-btc'
csv_path = os.path.join(bucket,data_key)
df = wr.s3.read_csv(path=csv_path)
df.head()

,Unnamed: 0,author,body,created_utc,id,permalink,subreddit,created,d_
0,0,Wsemenske,There's many things in calculus that were know...,1668556719,iwj096r,/r/Bitcoin/comments/yw7g7f/if_the_identity_of_...,Bitcoin,1.668557e+09,"{'author': 'Wsemenske', 'body': ""There's many ..."
1,1,cc-test,&gt;People are going to understand what Bitcoi...,1668556659,iwj04b4,/r/Economics/comments/yw3w5s/banking_giants_an...,Economics,1.668557e+09,"{'author': 'cc-test', 'body': '&gt;People are ..."
2,2,tickleMyBigPoop,good for bitcoin,1668556642,iwj02wg,/r/neoliberal/comments/ywdrsn/banking_giants_a...,neoliberal,1.668557e+09,"{'author': 'tickleMyBigPoop', 'body': 'good fo..."
3,3,DayTradeTard,"I love MSTR, high prems (wheeling months ~6 st...",1668556630,iwj01wy,/r/options/comments/ywc5wo/60k_to_try_the_whee...,options,1.668557e+09,"{'author': 'DayTradeTard', 'body': 'I love MST..."
4,4,-__-_-__-_-__-,So the basic idea is just that everyone can se...,1668556619,iwj00zi,/r/Buttcoin/comments/ywcw6g/decentralized_is_a...,Buttcoin,1.668557e+09,"{'author': '-__-_-__-_-__-', 'body': 'So the b..."


In [14]:
df['sentiment'] = 0
df = df[['body','created_utc','sentiment']]
df.head()

,body,created_utc,sentiment
0,There's many things in calculus that were know...,1668556719,0
1,&gt;People are going to understand what Bitcoi...,1668556659,0
2,good for bitcoin,1668556642,0
3,"I love MSTR, high prems (wheeling months ~6 st...",1668556630,0
4,So the basic idea is just that everyone can se...,1668556619,0


In [17]:
dt_list = []
for i in df['created_utc']:
    dt_list.append(datetime.fromtimestamp(i))
df['datetime'] = dt_list
df.head()

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,body,created_utc,sentiment,datetime
0,There's many things in calculus that were know...,1668556719,0,2022-11-15 23:58:39
1,&gt;People are going to understand what Bitcoi...,1668556659,0,2022-11-15 23:57:39
2,good for bitcoin,1668556642,0,2022-11-15 23:57:22
3,"I love MSTR, high prems (wheeling months ~6 st...",1668556630,0,2022-11-15 23:57:10
4,So the basic idea is just that everyone can se...,1668556619,0,2022-11-15 23:56:59


In [19]:
df['text'] = ""
for i in range(len(df)):
    no_punc = strip_punctuation(str(df.iloc[i,0]))
    no_punc = no_punc.replace("bitcoin","")
    no_punc = no_punc.replace("btc","")
    df.iloc[i,4] = no_punc
df.head()

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,body,created_utc,sentiment,datetime,text
0,There's many things in calculus that were know...,1668556719,0,2022-11-15 23:58:39,There s many things in calculus that were know...
1,&gt;People are going to understand what Bitcoi...,1668556659,0,2022-11-15 23:57:39,gt People are going to understand what Bitcoi...
2,good for bitcoin,1668556642,0,2022-11-15 23:57:22,good for bitcoin
3,"I love MSTR, high prems (wheeling months ~6 st...",1668556630,0,2022-11-15 23:57:10,I love MSTR high prems wheeling months 6 st...
4,So the basic idea is just that everyone can se...,1668556619,0,2022-11-15 23:56:59,So the basic idea is just that everyone can se...


In [20]:
class SentimentClassifier(nn.Module):
    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-cased")
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
    def forward(self, input_ids, attention_mask):
        returned = self.bert(
        input_ids=input_ids,
        attention_mask=attention_mask)
        pooled_output = returned["pooler_output"]
        output = self.drop(pooled_output)
        return self.out(output)

In [21]:
def load_model(model_dir="s3://bert-btc/model.pth"):    
    model = SentimentClassifier(3).to(device)
    with open(model_dir, "rb") as f:
        model.load_state_dict(torch.load(f))
    return model.to(device)

In [22]:
class redditDataset(Dataset):
  def __init__(self, reviews, targets, tokenizer, max_len=300): #,):
    self.reviews = reviews
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len
  def __len__(self):
    return len(self.reviews)
  def __getitem__(self, item):
    review = str(self.reviews[item])
    target = self.targets[item]
    encoding = self.tokenizer.encode_plus(
      review,
      add_special_tokens=True,
      max_length=300,
      return_token_type_ids=False,
      padding= 'max_length',
      return_attention_mask=True,
      return_tensors='pt',
      truncation=True
    )
    return {
      'review_text': review,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

def create_data_loader(df, tokenizer, batch_size,max_len=300):
  ds = redditDataset(
    reviews=df.body.to_numpy(),
    targets=df.sentiment.to_numpy(),
    tokenizer=tokenizer,
    max_len=300)
  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=1
  )

In [23]:
BATCH_SIZE = 16
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
data_loader = create_data_loader(df, tokenizer, BATCH_SIZE, max_len=300)

In [24]:
def get_predictions(model, data_loader):
  model = model.eval()
  review_texts = []
  predictions = []
  prediction_probs = []
  real_values = []
  with torch.no_grad():
    for d in data_loader:
      texts = d["review_text"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)
      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)
      review_texts.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(outputs)
      real_values.extend(targets)
  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  real_values = torch.stack(real_values).cpu()
  return review_texts, predictions, prediction_probs, real_values

In [25]:
trained_model = SentimentClassifier(3)
trained_model.load_state_dict(torch.load("model.pth",map_location=torch.device('cpu')))
trained_model = trained_model.to(device)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [26]:
y_review_texts, y_pred, y_pred_probs, y_test = get_predictions(trained_model,data_loader)
df['prediction'] = y_pred
df.head()

,body,created_utc,sentiment,datetime,text,prediction
0,There's many things in calculus that were know...,1668556719,0,2022-11-15 23:58:39,There s many things in calculus that were know...,2
1,&gt;People are going to understand what Bitcoi...,1668556659,0,2022-11-15 23:57:39,gt People are going to understand what Bitcoi...,2
2,good for bitcoin,1668556642,0,2022-11-15 23:57:22,good for bitcoin,0
3,"I love MSTR, high prems (wheeling months ~6 st...",1668556630,0,2022-11-15 23:57:10,I love MSTR high prems wheeling months 6 st...,2
4,So the basic idea is just that everyone can se...,1668556619,0,2022-11-15 23:56:59,So the basic idea is just that everyone can se...,2


In [27]:
df['text'] = ""
for i in range(len(df)):
    no_punc = strip_punctuation(str(df.iloc[i,0]))
    no_punc = no_punc.replace("bitcoin","")
    no_punc = no_punc.replace("btc","")
    df.iloc[i,4] = no_punc
df.head()

,body,created_utc,sentiment,datetime,text,prediction
0,There's many things in calculus that were know...,1668556719,0,2022-11-15 23:58:39,There s many things in calculus that were know...,2
1,&gt;People are going to understand what Bitcoi...,1668556659,0,2022-11-15 23:57:39,gt People are going to understand what Bitcoi...,2
2,good for bitcoin,1668556642,0,2022-11-15 23:57:22,good for,0
3,"I love MSTR, high prems (wheeling months ~6 st...",1668556630,0,2022-11-15 23:57:10,I love MSTR high prems wheeling months 6 st...,2
4,So the basic idea is just that everyone can se...,1668556619,0,2022-11-15 23:56:59,So the basic idea is just that everyone can se...,2


In [29]:
df2 = df[['datetime','text','prediction']]
df2.head()

,datetime,text,prediction
0,2022-11-15 23:58:39,There s many things in calculus that were know...,2
1,2022-11-15 23:57:39,gt People are going to understand what Bitcoi...,2
2,2022-11-15 23:57:22,good for,0
3,2022-11-15 23:57:10,I love MSTR high prems wheeling months 6 st...,2
4,2022-11-15 23:56:59,So the basic idea is just that everyone can se...,2


In [31]:
write_WC_key = f"daily_wordcloud/df{yest}.csv"
write_WC_path = os.path.join(bucket,write_WC_key)
wr.s3.to_csv(df2,path=write_WC_path)

{'paths': ['s3://bert-btc/daily_wordcloud/df2022-11-15.csv'],
 'partitions_values': {}}

In [32]:
df=df[['datetime','created_utc','prediction']]

In [33]:
write_key =  f"daily_prediction/df{yest}.csv"
write_path = os.path.join(bucket,write_key)
wr.s3.to_csv(df,path=write_path)

{'paths': ['s3://bert-btc/daily_prediction/df2022-11-15.csv'],
 'partitions_values': {}}